In [146]:
import torch
import random
import math
import numpy as np
import pandas as pd
import torch.nn as nn
import lightning as L
import matplotlib.pyplot as plt

from torchmetrics import MetricCollection
from torchmetrics import MeanAbsoluteError
from torchmetrics import MeanSquaredError
from torchmetrics import R2Score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader, TensorDataset

def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    L.seed_everything(seed)

set_random_seed(1438)

Seed set to 1438


In [147]:
params_dict = {
    'feature_scaler': StandardScaler(),
    'label_scaler': StandardScaler(),
    'batch_size': 256,
    'net_architecture': [5,100,100,1],
    'activation_function': nn.ReLU,
    'optim_func': torch.optim.Adam,
    'lr': 0.001,
    'lr_factor':0.5,
    'lr_patience': 5,
    'lr_cooldown': 20,
}

# Подготовка данных

In [148]:
df = pd.read_csv('./data/clasdb_pi_plus_n.txt', delimiter='\t', header=None)
df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']
df.loc[8314:65671, 'Ebeam'] = 5.754 # peculiarity of this dataset.
df['phi'] = df.phi.apply(lambda x: math.radians(x))
df['cos_phi'] = df['phi'].apply(lambda x: math.cos(x))
df = df.iloc[df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']].drop_duplicates().index]
df = df.drop('id', axis=1)
df = df[df.dsigma_dOmega <= df.dsigma_dOmega.quantile(0.96)]
df = df[df['error'] <= df["error"].quantile(0.96)]
df = df.reset_index(drop=True)
df

,Ebeam,W,Q2,cos_theta,phi,dsigma_dOmega,error,cos_phi
0,1.515,1.11,0.3,0.793353,6.021386,2.1780,0.484013,0.965926
1,1.515,1.27,0.3,-0.793353,6.021386,2.8950,0.454498,0.965926
2,1.515,1.27,0.3,-0.923880,6.021386,2.4850,0.477752,0.965926
3,1.515,1.29,0.3,-0.382683,0.261799,2.2030,0.443164,0.965926
4,1.515,1.29,0.3,-0.382683,0.785398,4.1780,0.474797,0.707107
...,...,...,...,...,...,...,...,...
86104,5.499,2.01,4.0,0.975000,3.730641,0.1012,0.043165,-0.831470
86105,5.499,2.01,4.0,0.975000,3.992441,0.1199,0.076638,-0.659346
86106,5.499,2.01,4.0,0.975000,4.646939,0.1578,0.095391,-0.065403
86107,5.499,2.01,4.0,0.975000,4.777839,0.2346,0.158557,0.065403


In [149]:
feature_columns = ["Ebeam", "W",	"Q2",	"cos_theta", "cos_phi"]
feature_data = df[feature_columns]
label_data = df['dsigma_dOmega']


#TRAIN TEST SPLIT
X_train, X_residual, y_train, y_residual = train_test_split(feature_data,
                                                    label_data,
                                                    test_size=0.2,
                                                    random_state=42)

X_test, X_val, y_test, y_val = train_test_split(X_residual,
                                                y_residual,
                                                test_size=0.5,
                                                random_state=42)

In [150]:
#scale feature
scaler_feature = params_dict.get('feature_scaler')
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

columns_to_scale = list(X_train.columns)
X_train[columns_to_scale] = pd.DataFrame(scaler_feature.fit_transform(X_train[columns_to_scale]))
X_val[columns_to_scale] = pd.DataFrame(scaler_feature.transform(X_val[columns_to_scale]))
X_test[columns_to_scale] = pd.DataFrame(scaler_feature.transform(X_test[columns_to_scale]))

#scale target
scaler_target = params_dict.get('label_scaler')
y_train = pd.Series(scaler_target.fit_transform(y_train.to_frame())[:,0])
y_val = pd.Series(scaler_target.transform(y_val.to_frame())[:,0])
y_test = pd.Series(scaler_target.transform(y_test.to_frame())[:,0])

In [151]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
X_val = torch.tensor(X_val.values, dtype=torch.float32)

y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)

In [152]:
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_data, batch_size=params_dict.get('batch_size'), shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=params_dict.get('batch_size'), shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=params_dict.get('batch_size'), shuffle=False)

# Создание модели

In [153]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()

        self.net_architecture = params_dict.get('net_architecture')
        self.activation_function = params_dict.get('activation_function')

        self.network = nn.Sequential()
        for i in range(1,len(self.net_architecture)):
            self.network.append(nn.Linear(self.net_architecture[i-1], self.net_architecture[i]))
            if i!=len(self.net_architecture)-1:
                self.network.append(self.activation_function())

            else:
                pass

    def forward(self, x):
        return self.network(x)

In [154]:
model = NeuralNetwork()

# Обучение

In [155]:
class Pipeline(L.LightningModule):
    def __init__(self, model, params):
        super().__init__()

        self.model = model
        self.params = params
        self.criterion = torch.nn.MSELoss()
        self.optimizer = params_dict.get('optim_func')

        self.metrics = MetricCollection([
            MeanAbsoluteError(),
            MeanSquaredError(),
            R2Score()
        ])

        self.train_metrics = self.metrics.clone(postfix='/train')
        self.val_metrics = self.metrics.clone(postfix='/val')

    def configure_optimizers(self):
        optimizer = self.optimizer(self.parameters(), lr=self.params.get('lr'))

        lr_optim = ReduceLROnPlateau(optimizer = optimizer,
                                     mode = 'min',
                                     factor = self.params.get('lr_factor'),
                                     patience = self.params.get('lr_patience'),
                                     cooldown=self.params.get('lr_cooldown'),
                                     threshold=0.01
                                     )
        return {"optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": lr_optim,
                    "interval": "epoch",
                    "monitor": "val_loss",
                    "frequency": 2,
                    "name": 'lr_scheduler_monitoring'
                    },
                }

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = torch.sqrt(self.criterion(out.reshape(-1), y))
        self.train_metrics.update(out.reshape(-1), y)
        self.log("train_loss", loss, prog_bar=True)
        return loss


    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = torch.sqrt(self.criterion(out.reshape(-1), y))  #RMSE_Loss
        self.val_metrics.update(out.reshape(-1), y)
        self.log("val_loss", loss, prog_bar=True)

    def on_validation_epoch_end(self):
        self.log_dict(self.val_metrics.compute())
        self.val_metrics.reset()

    def on_train_epoch_end(self):
        self.log_dict(self.train_metrics.compute())
        self.train_metrics.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        self.metrics.update(out.reshape(-1), y)
        self.test_labels.append(y.cpu())
        self.test_predictions.append(out.cpu())

    def on_test_start(self):
        self.test_labels = []
        self.test_predictions = []

    def on_test_epoch_end(self):
        self.log_dict(self.metrics.compute())
        self.metrics.reset()

        all_labels = torch.cat(self.test_labels)
        all_predictions = torch.cat(self.test_predictions)

        self.results_df = pd.DataFrame({
            'true_label': all_labels.numpy().flatten(),
            'prediction': all_predictions.numpy().flatten()
        })

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        out = self.model(x)
        return out

In [156]:
L.seed_everything(1438)

model = NeuralNetwork()
pl_model = Pipeline(model, params=params_dict)
trainer = L.Trainer(max_epochs=100)

trainer.fit(
    model=pl_model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

Seed set to 1438
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | NeuralNetwork    | 10.8 K | train
1 | criterion     | MSELoss          | 0      | train
2 | metrics       | MetricCollection | 0      | train
3 | train_metrics | MetricCollection | 0      | train
4 | val_metrics   | MetricCollection | 0      | train
-----------------------------------------------------------
10.8 K    Trainable params
0         Non-trainable params
10.8 K    Total params
0.043     Total estimated model params size (MB)
20        Modules in train mode
0         Modules in eval mode


/Users/golda/Library/Caches/pypoetry/virtualenvs/msu-interpol-6qN01YQ5-py3.14/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/golda/Library/Caches/pypoetry/virtualenvs/msu-interpol-6qN01YQ5-py3.14/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 270/270 [00:01<00:00, 257.30it/s, v_num=7, train_loss=0.433]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 270/270 [00:01<00:00, 266.98it/s, v_num=7, train_loss=0.594, val_loss=0.449]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 270/270 [00:01<00:00, 231.56it/s, v_num=7, train_loss=0.308, val_loss=0.418]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 270/270 [00:01<00:00, 227.09it/s, v_num=7, train_loss=0.457, val_loss=0.416]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 270/270 [00:01<00:00, 223.04it/s, v_num=7, train_loss=0.232, val_loss=0.398]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 270/270 [00:01<

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 270/270 [00:01<00:00, 197.57it/s, v_num=7, train_loss=0.157, val_loss=0.218]


# Результаты

In [157]:
predictions = trainer.predict(dataloaders=test_dataloader)
predictions = torch.cat(predictions, dim=0)
predictions = scaler_target.inverse_transform(predictions)
predictions

/Users/golda/Library/Caches/pypoetry/virtualenvs/msu-interpol-6qN01YQ5-py3.14/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:149: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.predict(ckpt_path='best')` to use the best model or `.predict(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at /Users/golda/Documents/Study/MSU_interpol/lightning_logs/version_7/checkpoints/epoch=99-step=27000.ckpt
Loaded model weights from the checkpoint at /Users/golda/Documents/Study/MSU_interpol/lightning_logs/version_7/checkpoints/epoch=99-step=27000.ckpt
/Users/golda/Library/Caches/pypoetry/virtualenvs/msu-interpol-6qN01YQ5-py3.14/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Co

Predicting DataLoader 0: 100%|██████████| 34/34 [00:00<00:00, 497.90it/s]


array([[0.24110802],
       [0.34002287],
       [0.90078171],
       ...,
       [0.74839994],
       [0.438632  ],
       [0.22329707]], shape=(8611, 1))

In [158]:
y_test = y_test.reshape(-1, 1)
y_test = scaler_target.inverse_transform(y_test)
y_test

array([[0.15873   ],
       [0.39740001],
       [0.7077    ],
       ...,
       [0.68493   ],
       [0.4209    ],
       [0.22423002]], shape=(8611, 1))

In [145]:
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print(f"Mean Absolute Error: {mae:.5f}")
print(f"Mean Squared Error: {mse:.5f}")

Mean Absolute Error: 0.08342
Mean Squared Error: 0.01543


In [132]:
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print(f"Mean Absolute Error: {mae:.5f}")
print(f"Mean Squared Error: {mse:.5f}")

Mean Absolute Error: 0.08458
Mean Squared Error: 0.01520


In [108]:
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print(f"Mean Absolute Error: {mae:.5f}")
print(f"Mean Squared Error: {mse:.5f}")

Mean Absolute Error: 0.08399
Mean Squared Error: 0.01559


In [89]:
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print(f"Mean Absolute Error: {mae:.5f}")
print(f"Mean Squared Error: {mse:.5f}")

Mean Absolute Error: 0.08110
Mean Squared Error: 0.01454
